In [29]:
source("robust_lasso.R")

# 加载测试框架
library(testthat)

# 模拟数据
set.seed(123)
n <- 100
p <- 50
X <- matrix(rnorm(n * p), nrow = n, ncol = p)
y <- rnorm(n)
lambda <- 0.05

result <- robust_lasso(X, y, lambda = 0.05,method='CGDA', priority = 'speed')
print(result)



Warning message in robust_lasso(X, y, lambda = 0.05, method = "CGDA", priority = "speed"):
"The priority argument is only used when method is set to 'auto'."


$method
[1] "CGDA"

$fit
$fit$beta
 [1]  0.000000000  0.000000000  0.119849625  0.000000000  0.000000000
 [6]  0.000000000  0.000000000  0.007392976 -0.018663902  0.022725695
[11]  0.000000000  0.000000000  0.074010640  0.000000000  0.000000000
[16]  0.000000000  0.000000000  0.266634386  0.000000000  0.000000000
[21]  0.007300960 -0.029613100  0.000000000  0.197686282  0.000000000
[26]  0.127538424  0.000000000  0.000000000 -0.097685448 -0.021224654
[31]  0.000000000  0.000000000 -0.050548256  0.000000000 -0.033279025
[36] -0.074218144  0.000000000  0.000000000  0.000000000  0.000000000
[41]  0.071709157  0.000000000 -0.047577409 -0.037304917  0.000000000
[46]  0.049066217  0.000000000  0.000000000  0.000000000  0.000000000

$fit$iter
[1] 17

$fit$convergence
[1] TRUE




In [ ]:
test_that("robust_lasso works correctly with manual methods", {
  # 测试指定算法
  methods <- c("Coordinate_Descent", "LARS", "FISTA", "SLA", "CGDA", "ISTA", "PFA")
  for (method in methods) {
    result <- robust_lasso(X, y, lambda = 0.05, method = method)
    expect_equal(result$method, method)
    expect_true(!is.null(result$fit))
  }
})

In [ ]:
test_that("robust_lasso handles invalid methods gracefully", {
  # 测试未知算法
  expect_error(robust_lasso(X, y, lambda = 0.05, method = "unknown"), "Unknown method")
})
